## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.models import Model
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

import os
import itertools

# Disable GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

os.environ["CUDA_VISIBLE_DEVICE"] = "0"

Using TensorFlow backend.


In [3]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練的 epochs 數量

freeze_layers = 2 # 凍結網路層數

In [4]:
# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
resnet_model = ResNet50(
    input_shape=x_train.shape[1:],
    weights='imagenet', # 使用 imagenet 訓練好的參數來初始化
#     pooling="avg", # 代表最後一層使用Global Average Pooling
    include_top=False # 代表將原本的Dense layer拔掉，必須替換成自己的Dense layer
)

W0814 10:43:37.002923 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0814 10:43:37.509252 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0814 10:43:37.785706 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0814 10:43:37.960989 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0814 10:43:37.962969 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\backend\te

94658560/94653016 [==============================] - 21s 0us/step


In [6]:
x = resnet_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)

output_layer = Dense(num_classes, activation='softmax', name='output')(x)

model = Model(inputs=resnet_model.input, outputs=output_layer)
model.summary()

W0814 10:44:29.746160 71640 deprecation.py:506] From D:\python\lib\site-packages\keras\backend\tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [7]:
# 設定凍結與要進行訓練的網路層
for layer in model.layers[:freeze_layers]:
    layer.trainable = False
for layer in model.layers[freeze_layers:]:
    layer.trainable = True

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5),
              metrics=['accuracy'])

W0814 10:45:38.099996 71640 deprecation_wrapper.py:119] From D:\python\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
data_generator = ImageDataGenerator(
    rotation_range=10, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True)

history = model.fit_generator(
    # ImageDataGenerator
    data_generator.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0814 10:45:51.878427 71640 deprecation.py:323] From D:\python\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/30
390/390 [==============================] - 266s 681ms/step - loss: 3.2707 - acc: 0.1400 - val_loss: 2.2561 - val_acc: 0.2257
Epoch 2/30
390/390 [==============================] - 202s 519ms/step - loss: 2.2601 - acc: 0.2330 - val_loss: 1.9741 - val_acc: 0.3521
Epoch 3/30
390/390 [==============================] - 203s 520ms/step - loss: 2.0345 - acc: 0.3241 - val_loss: 1.7288 - val_acc: 0.4420
Epoch 4/30
390/390 [==============================] - 205s 526ms/step - loss: 1.8208 - acc: 0.4096 - val_loss: 1.5229 - val_acc: 0.5107
Epoch 5/30
390/390 [==============================] - 206s 527ms/step - loss: 1.6141 - acc: 0.4805 - val_loss: 1.3338 - val_acc: 0.5659
Epoch 6/30
390/390 [==============================] - 206s 528ms/step - loss: 1.4564 - acc: 0.5334 - val_loss: 1.2352 - val_acc: 0.6035
Epoch 7/30
390/390 [==============================] - 210s 539ms/step - loss: 1.3239 - acc: 0.5758 - val_loss: 1.1271 - val_acc: 0.6331
Epoch 8/30
390/390 [============================